In [35]:
API_KEY = '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'
from typing import TypedDict, Literal, Annotated
from langgraph.graph.message import add_messages
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode, tools_condition

In [44]:
llm = ChatGroq(api_key=API_KEY,model_name="llama-3.3-70b-versatile")
class State(TypedDict):
  messages: Annotated[list, add_messages]

def chatbot(state: State) -> State:
  # Append the new message to the list of messages
  return {"messages": state["messages"] + [llm.invoke(state["messages"])]}

builder = StateGraph(State)
builder.add_node("chatbot_node", chatbot)
builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node", END)
graph = builder.compile()
#message = {"role": "user", "content": "Hello, World!"}
#response = graph.invoke({"messages": [message]})
#response["messages"]

In [45]:
state = None
while True:
  in_message = input("You: ")
  if in_message.lower() in {"quit", "exit"}:
    break
  if state is None:
    state: State = {
        "messages": [{"role": "user", "content": in_message}]
    }
  else:
    state["messages"].append({"role": "user", "content": in_message})

  state = graph.invoke(state)
  print("Bot:", state["messages"][-1].content)

You: Hi
Bot: It's nice to meet you. Is there something I can help you with or would you like to chat?


KeyboardInterrupt: Interrupted by user